# Plotting the output from Cumulative event counter

Notebook just for compiling raw listed cell/event counter data as heatmaps

In [1]:
import csv, glob, re, os, json
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
from tqdm import tqdm
import render
import calculate_radial_analysis as calculate

# Compile bulk analysis

N_cells, N_events, P_events, cropped versions, comparative versions with colour bar limits, coefficient of variation calculations, a statistical relevance calculation based on that and finally a combined P_events above background (probability - control probability)

### Set parameters

In [ ]:
# t_range = 800
# radius = 800
# num_bins = 10
input_dict = {'focal_cell':'scr', 'focal_event':'div', 'subject_cell':'wt', 'subject_event':'div', 'radius': 800, 't_range': 800, 'num_bins': 10}
save_parent_dir = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_{}.{}.{}'.format(radius, t_range, num_bins)
input_dict

### Either compile from raw lists

In [ ]:
for config in ['800.800.20']:#['800.800.6', '600.600.8', '600.600.6', '500.500.6' '1000.1000.10', '1000.1000.6']:
    radius = int(config.split('.')[0])
    t_range = int(config.split('.')[1])
    num_bins = int(config.split('.')[2])

    ## canon
    raw_files_dir = os.path.join('/home/nathan/data/results/radial_analysis_output/follow/raw_lists/800.800/scr_div_wt_div/800.800')
    N_cells, N_events, P_events = render.cumulative_kymo_compiler(raw_files_dir, radius, t_range, num_bins)
    limit = np.amax(P_events)
    N = int(len(os.listdir(raw_files_dir))/2)

    ### control 
    # raw_files_dir = os.path.join('/home/nathan/data/results/radial_analysis_output/follow/raw_lists/800.800/wt_control_wt_div/800.800')
    # N_cells_c, N_events_c, P_events_c = render.cumulative_kymo_compiler(raw_files_dir, radius, t_range, num_bins)
    # limit_c = np.amax(P_events_c)
    # N_c = int(len(os.listdir(raw_files_dir))/2)

    ### save out raw arrays for coeff var
    save_parent_dir = '/home/nathan/data/results/radial_analysis_output/follow/arrays/'
    fn = os.path.join(save_parent_dir,'scr_div_wt_div_N_cell_N_event_P_event_{}.{}.{}.npz'.format(radius,t_range, num_bins))
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    # fn_c = os.path.join(save_parent_dir, 'arrays/control_ninety_ten_N_cell_N_event_P_event_{}.{}.{}.npz'.format(radius,t_range, num_bins))
    # if not os.path.exists(os.path.dirname(fn_c)):
    #     os.makedirs(os.path.dirname(fn_c))
    np.savez(fn, N_cells, N_events, P_events)
    # #np.savez(fn_c, N_cells_c, N_events_c, P_events_c) ## already previously saved control analysis

### Or load from previously compiles .npz stacks of N_cells, N_events, P_events

In [3]:
### load npz
with np.load('/home/nathan/data/results/radial_analysis_output/follow/arrays/scr_div_wt_div_N_cell_N_event_P_event_800.800.10.npz') as data:
    N_cells = data['arr_0']
    N_events = data['arr_1']
    P_events = data['arr_2']
with np.load('/home/nathan/data/results/radial_analysis_output/follow/arrays/control_N_cell_N_event_P_event_800.800.10.npz') as data:
    N_cells_c = data['arr_0']
    N_events_c = data['arr_1']
    P_events_c = data['arr_2']
N_c = 10491
N = 748

In [4]:
input_dict = {'radius': 800, 't_range': 800, 'num_bins': 10}
input_dict['N_cells'] = N_cells 
input_dict['N_events'] = N_events
input_dict['P_events'] = P_events 
input_dict['N_cells_c'] = N_cells_c 
input_dict['N_events_c'] = N_events_c 
input_dict['P_events_c'] = P_events_c 
input_dict['N'], input_dict['N_c'] = N, N_c 
input_dict['save_parent_dir'] = '/home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_{}.{}.{}'.format(input_dict['radius'], input_dict['t_range'], input_dict['num_bins'])

In [ ]:
render.MEGAPLOT(input_dict)

# Iterating over several different scales with bulk output

In [5]:
for config in ['800.800.10','800.800.20']:
    radius = int(config.split('.')[0])
    t_range = int(config.split('.')[1])
    num_bins = int(config.split('.')[2])
    print('starting dimensions radius, t_range, num_bins:',radius, t_range, num_bins)
    save_parent_dir = '/home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_{}.{}.{}'.format(radius, t_range, num_bins)
    ### load npz
    with np.load('/home/nathan/data/results/radial_analysis_output/follow/arrays/scr_div_wt_div_N_cell_N_event_P_event_{}.{}.{}.npz'.format(radius, t_range, num_bins, radius, t_range, num_bins)) as data:
        N_cells = data['arr_0']
        N_events = data['arr_1']
        P_events = data['arr_2']
    with np.load('/home/nathan/data/results/radial_analysis_output/follow/arrays/control_N_cell_N_event_P_event_{}.{}.{}.npz'.format(radius, t_range, num_bins, radius, t_range, num_bins)) as data:
        N_cells_c = data['arr_0']
        N_events_c = data['arr_1']
        P_events_c = data['arr_2']
    N_c = 10491
    N = 748
    
    input_dict = {'radius': radius, 't_range': t_range, 'num_bins': num_bins}
    input_dict['N_cells'] = N_cells 
    input_dict['N_events'] = N_events
    input_dict['P_events'] = P_events 
    input_dict['N_cells_c'] = N_cells_c 
    input_dict['N_events_c'] = N_events_c 
    input_dict['P_events_c'] = P_events_c 
    input_dict['N'], input_dict['N_c'] = N, N_c 
    input_dict['save_parent_dir'] = '/home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_{}.{}.{}'.format(input_dict['radius'], input_dict['t_range'], input_dict['num_bins'])

    render.MEGAPLOT(input_dict)

starting dimensions radius, t_range, num_bins: 800 800 10
Thank you for choosing MEGAPLOT




/home/nathan/analysis/cell-comp-analysis/radial_analysis/render.py:604: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Plot saved at  /home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_800.800.10/uncrop_unlim/scr_apop_wt_div/Spatiotemporal dist. of probability of wild-type divisions 
 around Scribble apoptoses (N=748) 800.800.10.pdf


/home/nathan/analysis/cell-comp-analysis/radial_analysis/render.py:604: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Plot saved at  /home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_800.800.10/uncrop_unlim/scr_apop_wt_div/Spatiotemporal dist. of wild-type divisions 
 around Scribble apoptoses (N=748) 800.800.10.pdf


...

Plot saved at  /home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_800.800.20/labelled/CV/wt_cont_wt_div/Coefficient of variation of probability of wild-type divisions 
 around wild-type random time points (N=10491) 800.800.20.pdf


/home/nathan/analysis/cell-comp-analysis/radial_analysis/render.py:604: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Plot saved at  /home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_800.800.20/labelled/CV/scr_apop_wt_div/Statisticall relevant areas of probability of wild-type divisions 
 around Scribble apoptoses (N=748) 800.800.20.pdf


/home/nathan/analysis/cell-comp-analysis/radial_analysis/render.py:604: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Plot saved at  /home/nathan/data/results/radial_analysis_output/follow/cumulative_plots/scr_div_wt_div_800.800.20/labelled/combined/scr_apop_wt_div/Difference in probability between 
canonical and control analysis 
i.e. probability of division above background 800.800.20.pdf
Plots saved out


In [ ]:
render.MEGAPLOT(N_cells, N_events, P_events, N_cells_c, N_events_c, P_events_c, N, N_c, limit, limit_c, cbar_lim, radius, t_range, num_bins, save_parent_dir)

In [ ]:
render.auto_plot_cumulative(
        P_events,
        "P_events",
        N,
        num_bins,
        radius,
        t_range,
        'scr',
        'div',
        'wt',
        'div',
        '',
        "",  # cbar_lim = (0, 0.002),
        include_apop_bin=True,
        bin_labels=False,
        SI=True,)
plt.show()

In [ ]:
plt.imshow(N_cells)
plt.show()

### Segregating raw lists based on experiment type

In [ ]:
ninety_ten = [
# 'GV0794/Pos3',
 'GV0794/Pos8',
 #'GV0794/Pos15',
 'GV0795/Pos3',
 'GV0795/Pos8',
 'GV0795/Pos16',
 'GV0796/Pos3',
 'GV0796/Pos8',
 'GV0796/Pos19',
 'GV0796/Pos20',
 'GV0797/Pos3',
 'GV0797/Pos8',
 'GV0797/Pos18',
 #'GV0797/Pos19',
 'GV0798/Pos3',
 'GV0798/Pos8',
 'GV0798/Pos18',
 'GV0798/Pos19',
 'GV0800/Pos3',
 'GV0800/Pos8',
 'GV0800/Pos15',
 'GV0801/Pos3',
 'GV0802/Pos3',
 #'GV0804/Pos3',
 'GV0805/Pos3',
 'GV0806/Pos3',
 'GV0807/Pos3']

In [ ]:
import shutil

In [ ]:
for fn in os.listdir('/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/raw_lists/1600.1600/control'):
    expt_ID = fn.split('_')[0]+'/'+fn.split('_')[1]
    if expt_ID in ninety_ten:
        shutil.copyfile(os.path.join('/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/raw_lists/1600.1600/control', fn),os.path.join('/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/raw_lists/1600.1600/control_ninety_ten', fn))